# Guinea-Bissau Demographics
---
Glenn Abastillas | April 7, 2020 | Visualizing Guinea-Bissau's Population

In [6]:
from pathlib import Path
from glob import glob
import pandas as pd
import geopandas as gpd
import altair as alt
import numpy as np
import json

datasets = Path("../data/datasets")
maps = Path("../data/maps")

### Source Data
The source data used for this page is a combination of Guinea-Bissau geographic shape files, census data, and preliminary disease data.

In [25]:
shapes = {'regions' : gpd.read_file(glob((maps / 'regions/*.shp').as_posix())[0])[['ADM1_NAME', 'ADM2_NAME', 'geometry']],
          'roads' : gpd.read_file(glob((maps / 'roads/*.shp').as_posix())[0]),
          'data' : pd.read_csv(glob((datasets / 'demographics/gpw*.csv').as_posix())[0])}

Merge data

In [30]:
exclude = ['GUBID', 'ISOALPHA', 'COUNTRYNM', 'NAME1', 
           'NAME2', 'NAME3', 'NAME4', 'NAME5', 'NAME6', 
           'CENTROID_X', 'CENTROID_Y', 'INSIDE_X', 'INSIDE_Y',]

In [31]:
data = shapes['regions'].merge(shapes['data'], how='left', left_on=['ADM1_NAME', 'ADM2_NAME'], right_on=['NAME1', 'NAME2'])
data = data.loc[:, ~data.columns.isin(exclude)]

In [36]:
# data.to_csv(datasets / 'demographics/demographics.csv', index=False)

Convert to `geojson` format for visualization.

In [34]:
geojson = json.loads(data.to_json())
geodata = alt.Data(values=geojson['features'])

Adjustable visualization settings

In [37]:
settings = {'width' : 800 , 'height' : 600}

chart = alt.Chart(geodata, **settings)

# Base maps
base = chart.mark_geoshape(fill='lightgray', stroke='black')
text = chart.mark_text(baseline='middle')
points = chart.mark_circle()

# Interactivity Controls
mouseover = alt.selection_single(on='mouseover', nearest=False)

# Basic Encoding Settings
color = alt.Color('properties.ADM1_NAME:N', legend=alt.Legend(title='Region'))
opacity = alt.condition(mouseover, alt.value(1), alt.value(0.5))
tooltip = [alt.Tooltip('properties.ADM1_NAME:N', title='Region'),
           alt.Tooltip('properties.ADM2_NAME:N', title='Sector'), 
           alt.Tooltip('properties.B_2010_E:Q', title='2010 Pop.', format=',.6')]

map_options = {'color' : color, 'tooltip' : tooltip, 'opacity' : opacity}

## Maps

The following maps display the distribution of the population by gender and age.

#### Guinea-Bissau by Gender

In [40]:
# gender_all = chart.mark_geoshape(stroke='black', strokeWidth=1)

base.encode(**map_options).add_selection(mouseover) #+ \
# points.encode(longitude='properties.CENTROID_X:Q', latitude='properties.CENTROID_Y:Q', color=alt.value('black'))
# text.encode(longitude='properties.CENTROID_X:Q', latitude='properties.CENTROID_Y:Q', color=alt.value('black')).encode(alt.Text('properties.ADM2_NAME:N'))

alt.Chart(...)

In [ ]:
Demographics